In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")


In [ ]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

NameError: ignored

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
class CustomDataset(Dataset):

    def __init__(self, dataframe, source_col, target_col, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_col = source_col
        self.target_col = target_col
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source = str(self.data[self.source_col][index])
        target = str(self.data[self.target_col][index])

        source = self.tokenizer.encode_plus(
            source,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt"
        )

        target = self.tokenizer.encode_plus(
            target,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt"
        )

        return {
            "input_ids": source["input_ids"].flatten(),
            "attention_mask": source["attention_mask"].flatten(),
            "labels": target["input_ids"].flatten(),
        }

from sklearn.model_selection import train_test_split
data = pd.read_excel(r'/content/drive/MyDrive/bart/MeQSum_ACL2019_BenAbacha_Demner-Fushman.xlsx')

train_data, temp = train_test_split(data, test_size=0.2, random_state=42)
test_data, val_data = train_test_split(temp, test_size=0.5, random_state=42)

train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)


train_dataset = CustomDataset(train_data, 'CHQ', 'Summary', tokenizer, 512)
val_dataset = CustomDataset(val_data, 'CHQ', 'Summary', tokenizer, 512)

In [ ]:
import numpy as np
best_loss = np.inf
best_batchsize = None
best_epoch = None

for batchsize in range(4,13,4):
    torch.cuda.empty_cache()
    train_dataloader = DataLoader(train_dataset, batch_size=batchsize)
    val_dataloader = DataLoader(val_dataset, batch_size=batchsize)

    optimizer = AdamW(model.parameters(), lr=1e-5)
    total_steps = len(train_dataloader) * 2
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(2):
        print(f'Epoch: {epoch+1}')

        # Training Phase
        model.train()
        total_train_loss = 0
        for batch in train_dataloader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()

        # Validation Phase
        model.eval()
        total_val_loss = 0
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f'Validation Loss: {avg_val_loss}')

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            best_batchsize = batchsize
            best_epoch = epoch + 1
            model_path = "/content/drive/MyDrive/bart_research/best_model"
            model.save_pretrained(model_path)
            print(f'Saving model at epoch {best_epoch} with validation loss of {best_loss:.3f}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1
Validation Loss: 0.68969717502594
Saving model at epoch 1 with validation loss of 0.690
Epoch: 2
Validation Loss: 0.33379624128341673
Saving model at epoch 2 with validation loss of 0.334
Epoch: 1
Validation Loss: 0.13006678968667984
Saving model at epoch 1 with validation loss of 0.130
Epoch: 2
Validation Loss: 0.09705775460371605
Saving model at epoch 2 with validation loss of 0.097
Epoch: 1


OutOfMemoryError: ignored

In [ ]:
model_path = "/content/drive/MyDrive/bart_research/best_model"
model=BartForConditionalGeneration.from_pretrained(model_path)
test_dataset = CustomDataset(test_data, 'CHQ', 'Summary', tokenizer, 1000)
test_dataloader = DataLoader(test_dataset, batch_size=8)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()
predictions = []

for batch in test_dataloader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs]
    predictions.extend(preds)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
print(predictions)

['What are the treatments for Beta Thalassemia Major?', 'What are the  treatments  for  giant  cell arteritis?', 'What are the treatments for shingles?', 'What are the treatments for Rubella?', 'Where can I find information on Ochoa syndrome?', 'What are the treatments for HYPOMATURATION?', 'What are the treatments for hoe in lung?', 'What are the treatments for obesity in Cambodia? In the USA and elsewhere, there are', 'What are the treatments for type 1 diabetes?', 'What are the treatments for a fractured nose?', 'What are the treatments for arm pain?', 'What are the treatments for lipoma of forearm?', 'What are the ingredients of Tolterodine Tartrate ER 4mg?', 'What are the effects of clonidine on a three year old female?', 'How can chronic myeloid leukemia (CML) be cured completely?', 'What are the treatments for hemitary angioedema?', 'Where can I find information on Tetanus immunity?', 'Where can I find information on muscle waste?', 'What are the treatments for esophygeal candid

In [ ]:
path='/content/drive/MyDrive/vocabulary.txt'
with open(path, "r") as file:
    vocabulary = file.readlines()

# Remove the newline character from each word
vocabulary = [word.strip() for word in vocabulary]


In [ ]:
from torch.utils.data import Dataset
import torch

class CustomDataset(Dataset):

    def __init__(self, dataframe, source_col, target_col, tokenizer, max_len, vocabulary):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_col = source_col
        self.target_col = target_col
        self.max_len = max_len
        self.vocabulary = vocabulary

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source = str(self.data[self.source_col][index])
        target = str(self.data[self.target_col][index])

        source = self.tokenizer.encode_plus(
            source,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt"
        )

        target = self.tokenizer.encode_plus(
            target,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt"
        )

        loss_mask = torch.ones(target["input_ids"].shape)
        for i, token_id in enumerate(target["input_ids"].flatten()):
            if self.tokenizer.decode(token_id) in self.vocabulary:
                loss_mask[0][i] = 2.0

        return {
            "input_ids": source["input_ids"].flatten(),
            "attention_mask": source["attention_mask"].flatten(),
            "labels": target["input_ids"].flatten(),
            "loss_mask": loss_mask  # Add the loss mask to the output
        }

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_excel(r'/content/drive/MyDrive/bart/MeQSum_ACL2019_BenAbacha_Demner-Fushman.xlsx')

train_data, temp = train_test_split(data, test_size=0.2, random_state=42)
test_data, val_data = train_test_split(temp, test_size=0.5, random_state=42)

train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)


train_dataset = CustomDataset(train_data, 'CHQ', 'Summary', tokenizer, 512, vocabulary)
val_dataset = CustomDataset(val_data, 'CHQ', 'Summary', tokenizer, 512, vocabulary)

In [ ]:
import numpy as np
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
import torch

best_loss = np.inf
best_batchsize = None
best_epoch = None

for batchsize in range(4, 13, 4):
    torch.cuda.empty_cache()
    train_dataloader = DataLoader(train_dataset, batch_size=batchsize)
    val_dataloader = DataLoader(val_dataset, batch_size=batchsize)

    optimizer = AdamW(model.parameters(), lr=1e-5)
    total_steps = len(train_dataloader) * 2
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(2):
        print(f'Epoch: {epoch+1}')

        # Training Phase
        model.train()
        total_train_loss = 0
        for batch in train_dataloader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            loss_mask = batch["loss_mask"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Scale the loss for special tokens
            loss = loss * loss_mask
            loss = loss.sum() / loss_mask.sum()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()

        # Validation Phase
        model.eval()
        total_val_loss = 0
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f'Validation Loss: {avg_val_loss:.4f}')

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            best_batchsize = batchsize
            best_epoch = epoch + 1
            model_path = "/content/drive/MyDrive/bart_ft_research/best_model"
            model.save_pretrained(model_path)
            print(f'Saving model at epoch {best_epoch} with validation loss of {best_loss:.4f}')
        else:
            print(f'Validation loss did not improve from {best_loss:.4f}')

        print('-' * 60)

print(f'Best Model saved with Batch Size: {best_batchsize} and Epoch: {best_epoch}')
print(f'Best Validation Loss: {best_loss:.4f}')



Epoch: 1


In [ ]:
!pip install rouge


In [ ]:
predictions = ["What precautions should a parent of a child with type 1 diabetes take when Pseudomonas aeruginosa is detected in their building's water supply?"]
reference_summaries=["Are there special precautions for diabetes and pseudomonas aeruginosa?"]
from rouge import Rouge

rouge = Rouge()
rouge_l_scores = rouge.get_scores(predictions, reference_summaries, avg=True)['rouge-l']

print("ROUGE-L Scores:", rouge_l_scores)

ROUGE-L Scores: {'r': 0.2222222222222222, 'p': 0.09090909090909091, 'f': 0.12903225394380866}


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')
# Tokenize the reference summaries and predictions
tokenized_reference_summaries = [word_tokenize(ref) for ref in reference_summaries]
tokenized_predictions = [word_tokenize(pred) for pred in predictions]

# Compute METEOR scores
meteor_scores = [meteor_score([ref], pred) for ref, pred in zip(tokenized_reference_summaries, tokenized_predictions)]
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)

print("Average METEOR Score:", avg_meteor_score)


Average METEOR Score: 0.3234782608695652


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Format the reference summaries for use with nltk's corpus_bleu
references = [[ref.split()] for ref in reference_summaries]
# Tokenize the generated predictions
candidates = [pred.split() for pred in predictions]

bleu_score = corpus_bleu(references, candidates)

print("BLEU Score:", bleu_score)


BLEU Score: 9.893133360884868e-232


In [ ]:
class T5CustomDataset(Dataset):

    def __init__(self, dataframe, source_col, target_col, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_col = source_col
        self.target_col = target_col
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source = "summarize: " + str(self.data[self.source_col][index])
        target = str(self.data[self.target_col][index])

        source = self.tokenizer.encode_plus(
            source,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt",
             truncation=True
        )

        target = self.tokenizer.encode_plus(
            target,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt",
             truncation=True
        )

        return {
            "input_ids": source["input_ids"].flatten(),
            "attention_mask": source["attention_mask"].flatten(),
            "labels": target["input_ids"].flatten(),
        }

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
train_dataset_t5 = T5CustomDataset(train_data, 'CHQ', 'Summary', tokenizer, 1000)
val_dataset_t5 = T5CustomDataset(val_data, 'CHQ', 'Summary', tokenizer, 1000)
test_dataset_t5 = T5CustomDataset(test_data, 'CHQ', 'Summary', tokenizer, 1000)

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
import numpy as np
best_loss = np.inf
best_batchsize = None
best_epoch = None

for batchsize in range[4,9,4]:
    torch.cuda.empty_cache()
    train_dataloader = DataLoader(train_dataset_t5, batch_size=batchsize)
    val_dataloader = DataLoader(val_dataset_t5, batch_size=batchsize)

    optimizer = AdamW(model.parameters(), lr=1e-5)
    total_steps = len(train_dataloader) * 2
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(2):
        print(f'Epoch: {epoch+1}')

        # Training Phase
        model.train()
        total_train_loss = 0
        for batch in train_dataloader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()

        # Validation Phase
        model.eval()
        total_val_loss = 0
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f'Validation Loss: {avg_val_loss}')

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            best_batchsize = batchsize
            best_epoch = epoch + 1
            model_path = "/content/drive/MyDrive/t5/best_model"
            model.save_pretrained(model_path)
            print(f'Saving model at epoch {best_epoch} with validation loss of {best_loss:.3f}')

Epoch: 1


OutOfMemoryError: ignored

In [ ]:
model_path = "/content/drive/MyDrive/t5/best_model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
test_dataset_t5 = T5CustomDataset(test_data, 'CHQ', 'Summary', tokenizer, 700)
test_dataloader = DataLoader(test_dataset_t5, batch_size=8)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
model.eval()
predictions = []

for batch in test_dataloader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs]
    predictions.extend(preds)

In [ ]:
print(predictions)

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [ ]:
model_path = "/content/drive/MyDrive/t5/best_model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
test_dataloader = DataLoader(test_dataset_t5, batch_size=8)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()
predictions = []

for batch in test_dataloader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs]
    predictions.extend(preds)
reference_summaries = test_data['Summary'].tolist()
from rouge import Rouge

rouge = Rouge()
rouge_l_scores = rouge.get_scores(predictions, reference_summaries, avg=True)['rouge-l']

print("ROUGE-L Scores:", rouge_l_scores)

from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')
# Tokenize the reference summaries and predictions
tokenized_reference_summaries = [word_tokenize(ref) for ref in reference_summaries]
tokenized_predictions = [word_tokenize(pred) for pred in predictions]

# Compute METEOR scores
meteor_scores = [meteor_score([ref], pred) for ref, pred in zip(tokenized_reference_summaries, tokenized_predictions)]
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)

print("Average METEOR Score:", avg_meteor_score)

from nltk.translate.bleu_score import corpus_bleu

# Format the reference summaries for use with nltk's corpus_bleu
references = [[ref.split()] for ref in reference_summaries]
# Tokenize the generated predictions
candidates = [pred.split() for pred in predictions]

bleu_score = corpus_bleu(references, candidates)

print("BLEU Score:", bleu_score)

ValueError: ignored